In [31]:
# 载入
import json
input_file_path="/home/ckqsudo/code2024/0dataset/E-bed/Chinese/中文数据_处理错reasoning_分离长对话数据_分离label_对话人.json"
with open(input_file_path,"r",encoding="utf-8") as i_f:
    cn_data=json.load(i_f)

In [32]:
# dialog_id=0
cn_dialog_data=[]
for x,item in enumerate(cn_data):
    if item['key']=="fewshot":
        new_dialog_json=[]
        new_list={"unit_id":item["unit_id"],"dialog_list":[]}
        for y,dialog_item in enumerate(item['dialog']):
            # print(dialog_item)
            new_unit={"id":dialog_item["id"],"participant":dialog_item["participant"],"text":dialog_item["text"]}
            new_list['dialog_list'].append(new_unit)
            # new_list["dialog_list"]
        cn_dialog_data.append(new_list)

In [33]:
# 计算message 和string中的token数量
from pyexpat.errors import messages
import tiktoken
# 2. Load an encoding
# 使用tiktoken.get_encoding()按名称加载编码。
# 第一次运行时，它将需要互联网连接进行下载。后续运行不需要互联网连接。
encoding = tiktoken.get_encoding("cl100k_base")
# 使用tiktoken.encoding_for_model()函数可以自动加载给定模型名称的正确编码。
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
# 3. Turn text into tokens with encoding.encode()
# The .encode() method converts a text string into a list of token integers.
encoding.encode("tiktoken is great!")
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """返回文本字符串中的Token数量"""
    num_tokens=0
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens
def calculate_messages(messages:list) -> int:
    """返回总的message长度"""
    counter=0
    for mess in messages:
        counter+=num_tokens_from_string(mess.content,"cl100k_base")
    return counter
num_tokens_from_string(json.dumps(cn_data),"cl100k_base")
# 2554978
# 1937335
# 2541877

2554978

In [34]:
num_tokens_from_string(json.dumps(cn_dialog_data[168]['dialog_list'],ensure_ascii=False),"cl100k_base")

18269

In [35]:
len(cn_dialog_data[168]['dialog_list'])


98

In [36]:
print(len(cn_dialog_data))

462


In [37]:
from z6_prompt import *
# print(input_str) 
# from z6_prompt import *
import json

In [1]:
# from dotenv import load_dotenv
import os
os.environ['OPENAI_API_BASE'] ='https://gptnb.keqichen.top/v1'
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage,AIMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(temperature=0,model="gpt-3.5-turbo")
# long_chat =ChatOpenAI(temperature=0,model="gpt-3.5-turbo-16k")

In [2]:
# print(chat.openai_api_base)

https://gptnb.keqichen.top/v1


**********


In [39]:

# question=HumanMessage(content="请你帮我纠正下面这段心理咨询对话OCR后的结果，以json数组形式输入，以json数组的形式输出:"+json.dumps(cn_dialog_data[315]['dialog_list'],ensure_ascii=False))

# messages.append(question)

In [40]:
import os 
cache_dir_path="/home/ckqsudo/code2024/0dataset/E-bed/Chinese/ChatGPT_缓存文件夹"
def get_beg_idx(dir_path):
    files=os.listdir(dir_path)
    if len(files)==0:
        beg_idx=0
    else:
        idx_list=[]
        files.sort()
        for f in files:
            tmp=f.split("_")[-1].split(".")[0]
            idx_list.append(int(tmp))
        beg_idx=max(idx_list)
    return beg_idx
beg_idx=get_beg_idx(cache_dir_path)

In [41]:
beg_idx

461

In [42]:
messages = [
    SystemMessage(
        content="我希望你扮演一个中文语法和心理咨询专家，你需要根据语境纠正下列文本中的OCR的语法错误和中文符号使用错误。我希望你以json格式输出纠正错误OCR识别后的结果。json元素的键值包括：'id','participant','text','reason',其中'id','participant' 保留原始数据，不得修改，如果text中有语法或符号错误，输出修改后结果。'reason'记录修改原因，如果没有修改，reason值为空。请不要大幅度修改句子结构，主要修改错词和错误的中文标点符号。下面为部分例子，请确保输出格式为标准的json格式："
    ),
    HumanMessage(
        content=r"""请你帮我纠正下面这段心理咨询对话OCR后的结果，以json数组形式输入，以json数组的形式输出:"""+input_str
    ),
    AIMessage(content=output_str),
    HumanMessage(
        content=r"""请你帮我纠正下面这段心理咨询对话OCR后的结果，以json数组形式输入，以json数组的形式输出:"""+input_str2
    ),
    AIMessage(
        content=output_str2
    ),
    HumanMessage(content=r"""请你帮我纠正下面这段心理咨询对话OCR后的结果，以json数组形式输入，以json数组的形式输出:"""+input_str3),
    AIMessage(
        content=output_str3
    ),
    HumanMessage(content="请你帮我纠正下面这段心理咨询对话OCR后的结果，以json数组形式输入，以json数组的形式输出:"+json.dumps(cn_dialog_data[315]['dialog_list'],ensure_ascii=False))
]

In [43]:
len(messages),beg_idx

(8, 461)

In [44]:
beg_idx=get_beg_idx(cache_dir_path)

In [45]:
beg_idx

461

In [46]:
json.dumps(cn_dialog_data[170]['dialog_list'],ensure_ascii=False)

'[{"id": "170-0", "participant": "治疗师", "text": "说说看，都发生了什么？"}, {"id": "170-1", "participant": "来访者", "text": "真是太有收获了。Clark跟往常一样走进诊室，告诉我他都遭遇了哪些事。听他讲的那些故事，照说我应该感到很有意思才对，因为他谈了与供货商之间发生的争执，他设法与之周旋，表现得相当好：可是呢，我却发现那故事让我觉得挺无聊的。"}, {"id": "170-2", "participant": "治疗师", "text": "这果然很有意思。你怎么解释自己这种情绪呢？"}, {"id": "170-3", "participant": "来访者", "text": "我觉得在某种程度上，他从我身边“走开”了。这可能是他对上次被我伤害、被我误解所做的反应吧。"}, {"id": "170-4", "participant": "治疗师", "text": "的确是这样。当你对一个来访者感到无聊时，通常这不是面询内容本身所造成的。失去与来访者在情感上的联系，是引发疏离感的原因。从婴儿期开始，我们就在回应他人以各种方式传递给我们的信息，诸如说话声调、身体姿态，以及他人对我们的整体态度。当来访者退缩时，尽管他可能依然在不停地说，但语气语调都会有变化，似乎不再是对着我们说话，而像是在对着一堵白墙；对此，我们的反应往往是不知不觉中感到孤单、隔离与无意义一一换言之，感到被遗弃。假如对这些没有觉察，我们也会转而退缩，从来访者身边“走开”；哪怕我们在继续专注地看着对方，甚至装出很感兴趣的样子。"}, {"id": "170-5", "participant": "来访者", "text": "发生这种情况，你会怎么处理呢？"}, {"id": "170-6", "participant": "治疗师", "text": "不要让自己沉浸在闪问不乐中，不要陷于被动而无所作为，而要积极地与来访者互动。假如你还搞不清当前状况，那就去探寻看看。来访者沉默不语，这会是什么原因呢：咨询中他为什么会漫无目的，表现得随波逐流呢；或者，他在详述那些看起来无关紧要的细节，这是为什么呢？对此类现象，你无需犹豫，只管把你的假设说出来。而不要去想当然地作意断。假如来访者滔潘不绝地漫谈，你则切忌对此

In [47]:
[1]+[2],beg_idx

([1, 2], 461)

In [48]:
beg_idx=461

In [53]:
lists= [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
n = 3 # 表示多少个一组
list_ = [lists[i:i+n] for i in range(0,len(lists),n)]
print(list_)
for i in list_:
    print(i)

[[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11], [12]]
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
[9, 10, 11]
[12]


In [50]:
# beg_idx=0
import json
cache_dir="/home/ckqsudo/code2024/0dataset/E-bed/Chinese/ChatGPT_缓存文件夹"
error_idx=[168,170, 175, 183, 188, 251, 322, 411, 415, 418, 440]
print("单独处理特殊样本")
for x,dialog_item in enumerate(cn_dialog_data):
    if x>=beg_idx or x in error_idx:
        file_name="d"+"_"+str(x)+".json"
        print(file_name)
        with open(cache_dir+"/"+file_name,"w",encoding="utf-8") as o_f:
            d_list=dialog_item['dialog_list']
            messages[-1]=HumanMessage(content=instruct_prompt+json.dumps(d_list,ensure_ascii=False))# 获得的是一个列表
            print(type(d_list),d_list[0],len(d_list)/2)
            total_tokens=calculate_messages(messages)
            last_tokens=num_tokens_from_string(messages[-1].content,"cl100k_base")
            interval=10
            n_list=[d_list[i:i+interval] for i in range(0,len(d_list),interval)]
            # for item_list  这里好像不小心删掉了

            if last_tokens*2/3>4096 or total_tokens>4096-100:
                response=long_chat(messages).content
                response_json=json.loads(response)
            else:
                response=chat(messages).content
                response_json=json.loads(response)
            json.dump(response_json,o_f,ensure_ascii=False)
            

单独处理特殊样本
d_168.json
<class 'list'> {'id': '168-0', 'participant': '来访者', 'text': '你怎么理解这个案例呢？你的诊断是什么？'} 49.0
18308 20870


[
    {
        "id": "168-0",
        "participant": "来访者",
        "text": "你怎么理解这个案例呢？你的诊断是什么？",
        "reason": ""
    },
    {
        "id": "168-1",
        "participant": "治疗师",
        "text": "我认为我们不该在目前仅有的信息基础上，去给Clark先生下结论。我们可以等你下回和他再见过一次面后，再来谈谈，那时我们再看看走到了哪一步。而现在，我想继续刚才的话题，继续说说诊断过程，并且，我还可以给你举几个例子，具体阐明在实际临床治疗中，面对打破我们原先预期的情形，或者面对不常见的、令人困惑的情形，我们该如何去得出结论。Billy，9岁男孩，某次手臂骨折后送进医院。儿科医生偶然间得知，Billy在视听方面皆有幻觉，持续几个月了。医生初步诊断为精神分裂症，并提出应该对他做会诊。当我走进病房，听到电视里传来的隆隆响声，正在播放着儿童节目，我看到一个小男孩坐在床上，正沉浸于某种绘画游戏里。尽管他左臂还打着石膏，可他似乎操作得还是蛮成功的。我打招呼说：“嗨！”想把他的注意力请吸引到我这里来。他抬头看着我，眼神澄澈晶莹，他用友善的微笑回应着我。就在那一刻，我清楚地意识到自己已经下了个判断：他不是个患有精神分裂症的男孩。我低头核对了一下转诊单，想确认自己是否走错病房、见错病人。结果，一切都没有错。",
        "reason": ""
    },
    {
        "id": "168-2",
        "participant": "来访者",
        "text": "对不起，我打断一下。你不会是在说，直觉预感应该成为我们诊断的基础吧？你是这个意思么？",
        "reason": ""
    },
    {
        "id": "168-3",
        "participant": "治疗师",
        "text": "我不是在说“应该”或“不应该”，而是试着向你描述，在我和Billy互动时发生的真实情况。当然啦，我不会在此刻就止步的，我要继续完成对他的诊断。我做了自我介绍，然后和Billy漫谈了

JSONDecodeError: Unterminated string starting at: line 59 column 17 (char 3559)

In [ ]:

            # if last_tokens*2/3>16000 or total_tokens>16000:
            #     print(last_tokens,total_tokens)
            #     pre_half=d_list[:len(d_list)//2]
            #     messages[-1]=HumanMessage(content=instruct_prompt+json.dumps(pre_half,ensure_ascii=False))
            #     pre_res=chat(messages).content
            #     print(pre_res)
            #     pre_json=json.loads(pre_res)
            #     pre_json=None
            #     post_half=d_list[len(d_list)//2:]
            #     messages[-1]=HumanMessage(content=instruct_prompt+json.dumps(post_half,ensure_ascii=False))
            #     post_json=json.loads(chat(messages).content)
            #     post_json=None
            #     response_json=pre_json+post_json
            #     print(response_json)
            # el

In [20]:
# messages
# HumanMessage(content="请你帮我纠正下面这段心理咨询对话OCR后的结果，以json数组形式输入，以json数组的形式输出:"+json.dumps(cn_dialog_data[315]['dialog_list'],ensure_ascii=False))

NameError: name 'cn_dialog_data' is not defined

[1, 14, 21]

In [1]:
# testfiles=["d_1.json","d_21.json","d_14"]
# testfiles.sort()
# print(testfiles)

['d_1.json', 'd_14', 'd_21.json']
